# $\alpha$ ion-ion streaming instability - Davies method
Solve in ordinary precision arithmetic.  Allow complex $\omega$.

Here, use the Davies method with squircle contour to find roots of $D(\omega)$ for given parameters and wavevector $\mathbf{k}$.

*Note:* Sometimes many points are required on the contour (100,000).  Additionally, the loss of precision is definitely noticeable here, where the solved-for roots are not exact (and may need to be refined with a local Newton solver).

In [ ]:
import Parameters
import PyPlot as plt
import QuadGK   # Note.  QuadGK seems to work with BigFloat, but NOT ArbComplex
import Roots    # for find_zero.  Parts of it work with univariate complex functions
                #  Note.  Looks like Roots works with BigFloat, but NOT ArbComplex

using FromFile
@from "tools.jl" import tools
@from "plasma_dispersion_tools.jl" import plasma_dispersion_tools

pd = plasma_dispersion_tools

In [ ]:
# squircle parameters
a = 0.996  # squareness
Rx = 0.0   # center of recturcle in x  (omega_r)
rx = 4.6    # half-length of recturcle in x (omega_r)
Ry = 0.01  # with small epsilon, max of squircle in y goes to 2*Ry (omega_i)
epsilon = 1e-5
smallnum = 1e-4
params = tools.Sqparams(a, Rx, Ry, rx, epsilon, smallnum);

visualize = true
if visualize
    N = 500
    theta = tools.theta_circle(N)
    points_on_squircle = zeros(ComplexF64, N)
    for j = 1:N
        points_on_squircle[j] = tools.gamma(theta[j], params)
    end
    z = points_on_squircle;
    plt.plot(real(z), imag(z)) 
end

In [ ]:
# tau_e = 1
# tau_alpha = 0.001
# nu_alpha = 0.01
# nu_d = 0.5 - nu_alpha
# ui = 10
# k = 0.01
# #phi = 89.6 * pi/180
# phi = 20 * pi/180
# kparallelhat = k * cos(phi)
# kperphat = k * sin(phi)
# # instability at omega = 0.033 + 0.00026im

In [ ]:
tau_e = 1
tau_alpha = 0.001
nu_alpha = 0.01
nu_d = 0.5 - nu_alpha
ui = 20
k = 0.4
#phi = 89.6 * pi/180
phi = 83.6 * pi/180
kparallelhat = k * cos(phi)
kperphat = k * sin(phi)
#kparallelhat = 0.23 / 150
#kperphat = 0.93 / 150

function D(omegahat)
    D, Dprime = pd.DDprime_aiisi(omegahat, kparallelhat, kperphat, nu_d, nu_alpha, tau_e, tau_alpha, ui)
    return D
end

function Dprime(omegahat)
    D, Dprime = pd.DDprime_aiisi(omegahat, kparallelhat, kperphat, nu_d, nu_alpha, tau_e, tau_alpha, ui)
    return Dprime
end

function DDprime(omegahat)
    D, Dprime = pd.DDprime_aiisi(omegahat, kparallelhat, kperphat, nu_d, nu_alpha, tau_e, tau_alpha, ui)
    return D, Dprime
end

In [ ]:
Npts = 200000
num_zeros = tools.count_zeros_inside_sq_contour(D, Npts, params)
println("Number of zeros found: ", num_zeros)
if num_zeros != 0
    roots = tools.all_roots(DDprime, num_zeros, Npts, params)
end

## Refine the root

In [ ]:
println("Before refining: D(ω) = ", D(roots[2]))
newroot = Roots.find_zero(D, roots[2], Roots.Secant())
println(newroot)
println("After refining: D(ω) = ", D(newroot))